In [ ]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential
from keras.callbacks import TensorBoard
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [ ]:
import os
import numpy as np
#from keras.preprocessing.image import img_to_array, load_img
from PIL import Image

X = []
directory = '/content/'  # Replace this with the path to your image directory

# Loop through the files in the directory and load the images

image = img_to_array(load_img(os.path.join(directory, img_gray_version1.png)))
X.append(image)

X = np.array(X, dtype=float)

# Normalize the pixel values to the range [0, 1]
#X /= 255.0

# Set up train and test data
# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

NameError: ignored

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1, steps_per_epoch=10)

<ipython-input-10-e27311459831>:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1, steps_per_epoch=10)


10/10 [==============================] - 4s 62ms/step - loss: 0.0000e+00


In [ ]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [ ]:
from skimage.transform import resize

# Assuming your model expects input shape (None, 256, 256, 1)

# Resize the test data
Xtest_resized = np.array([resize(img, (256, 256, 1)) for img in Xtest])

# Evaluate the model
Xtest_resized = np.array([resize(img, (256, 256, 1)) for img in Xtest])


In [ ]:
# Test images
from skimage.transform import resize

Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
Xtest_resized = np.array([resize(img, (256, 256, 1)) for img in Xtest])
Xtest_resized = np.array([resize(img, (256, 256, 1)) for img in Xtest])

In [ ]:
from skimage.transform import resize
from skimage import img_as_ubyte
from skimage.exposure import rescale_intensity
color_me = []
directory = '/content/'  # Replace this with the path to your image directory

# Loop through the files in the directory and load the images
for filename in os.listdir(directory):
    # Check if the file is an image (supports common image file extensions)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        image = img_to_array(load_img(os.path.join(directory, filename)))
        color_me.append(image)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

color_me_resized = np.array([resize(img, (256, 256, 1)) for img in color_me])

# Test model
output = model.predict(color_me_resized)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    # Resize color_me to 256x256 to match the size of cur
    color_me_i_resized = resize(color_me[i], (256, 256), mode='constant').squeeze()
    cur[:, :, 0] = color_me_i_resized
    cur[:, :, 1:] = output[i]

    # Clip the color data to stay within [0, 1]
    cur = np.clip(cur, 0, 1)

    # Enhance contrast using contrast stretching for better visualization
    contrast_stretched = rescale_intensity(cur, in_range=(0.2, 0.8), out_range=(0, 1))

    # Convert the contrast-enhanced color data to uint8 format
    cur_uint8 = img_as_ubyte(contrast_stretched)

    # Save the image with full path
    save_path = os.path.join('result', f"img_{i}.png")

    # Create the 'result' directory if it doesn't exist
    if not os.path.exists('result'):
        os.makedirs('result')

    imsave(save_path, lab2rgb(cur_uint8))

1/1 [==============================] - 1s 532ms/step


<ipython-input-33-237823fd47a5>:47: UserWarning: result/img_0.png is a low contrast image
  imsave(save_path, lab2rgb(cur_uint8))
